In [1]:
!nvidia-smi

Wed Apr  3 20:06:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:19:00.0 Off |                  N/A |
| 30%   30C    P3             128W / 350W |     10MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# from accelerate import notebook_launcher
# from accelerate imporself.model_.t Accelerator

# def test_loop():
#     accelerator = Accelerator()
#     print(accelerator.state)
# notebook_launcher(test_loop, [], num_processes=2)

In [3]:
# Load model directly
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm
from datasets import load_dataset

import transformers
from tqdm.auto import tqdm, trange

from accelerate import notebook_launcher
from accelerate import Accelerator

import sys
sys.path.insert(1, '/home/usoltsev/study/repositories/rugpt-memory/')

# assert torch.cuda.is_available()
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from src.models.ltm_gpt.ltm_gpt import LTM_GPT
from src.utils.train_config import load_config
from src.models.load_base_model import load_base_model

In [5]:
def custom_generate(prompt, model, device, max_steps):
    batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    print(batch)

    for i in range(max_steps):
        outputs = model(**batch)
        # print(outputs)
        probs = outputs.logits[0, -1].nan_to_num(nan=0.0).div(0.8).softmax(-1)  # .argmax(-1).reshape(1, 1)
        old_token = outputs.logits[0, -1].argmax(-1).reshape(1, 1)
        # print(old_token)
        next_token = torch.multinomial(probs, 1).reshape(1, 1)
        # print(next_token)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)
        break

    return tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()[1:])

In [6]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import os
os.environ['CURL_CA_BUNDLE'] = ''

In [7]:
main_config = load_config('/home/usoltsev/study/repositories/rugpt-memory/configs/finetuning_codeparrot.yml')

model, tokenizer = load_base_model(main_config)

[2024-04-03 20:06:55,778: INFO] Loading base model (load_base_model.py:9)


/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 6/6 [00:21<00:00,  3.61s/it]
/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: Inse

In [8]:
assert torch.cuda.is_available()
device_for_ltm_layers = torch.device('cuda:1')

In [9]:
cnt_blocks_with_memory = 2

model = LTM_GPT(
    model,
    cnt_blocks_with_memory=cnt_blocks_with_memory,
    device=device_for_ltm_layers
)

In [10]:
for param in model.transformer.h[-cnt_blocks_with_memory:].parameters():
    param.requires_grad=True
    # param.data = param.data.to(torch.float32)

for param in model.transformer.ln_f.parameters():
    param.requires_grad=True
    # param.data = param.data.to(torch.float32)

for param in model.lm_head.parameters():
    param.requires_grad=True
    # param.data = param.data.to(torch.float32)

In [11]:
!nvidia-smi

Wed Apr  3 20:10:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:19:00.0 Off |                  N/A |
| 30%   30C    P8              21W / 350W |  12865MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) / sum(p.numel() for p in model.parameters())
count_parameters(model)

0.11794248348100973

# 2. Only LTM training. Data - JuliaWikiDataset

In [14]:
from src.data.wiki_dataset import WikiDataset
from src.data.wiki_dataloader import EpochDataloader

In [17]:
from pathlib import Path
dataset_path = '/home/usoltsev/study/repositories/rugpt-memory/data/dataset/'
n_context = 1

In [18]:
train_dataset = WikiDataset(data_path=Path(dataset_path),
                            n_context=n_context,
                            split='train')

Exception: Path to index does not exist! Try another one!

In [19]:
!ls /home/usoltsev/study/repositories/rugpt-memory/data/dataset/

wiki_00.jsonl  wiki_10.jsonl  wiki_20.jsonl  wiki_30.jsonl  wiki_40.jsonl
wiki_01.jsonl  wiki_11.jsonl  wiki_21.jsonl  wiki_31.jsonl  wiki_41.jsonl
wiki_02.jsonl  wiki_12.jsonl  wiki_22.jsonl  wiki_32.jsonl  wiki_42.jsonl
wiki_03.jsonl  wiki_13.jsonl  wiki_23.jsonl  wiki_33.jsonl  wiki_43.jsonl
wiki_04.jsonl  wiki_14.jsonl  wiki_24.jsonl  wiki_34.jsonl  wiki_44.jsonl
wiki_05.jsonl  wiki_15.jsonl  wiki_25.jsonl  wiki_35.jsonl  wiki_45.jsonl
wiki_06.jsonl  wiki_16.jsonl  wiki_26.jsonl  wiki_36.jsonl  wiki_46.jsonl
wiki_07.jsonl  wiki_17.jsonl  wiki_27.jsonl  wiki_37.jsonl
wiki_08.jsonl  wiki_18.jsonl  wiki_28.jsonl  wiki_38.jsonl
wiki_09.jsonl  wiki_19.jsonl  wiki_29.jsonl  wiki_39.jsonl


# 1. OutOfMemoryError: CUDA out of memory. Data - codeparrot_dataset

In [17]:
from src.data.load_codeparrot_dataset import load_codeparrot_dataset
from torch.utils.tensorboard import SummaryWriter
dataset = load_codeparrot_dataset(tokenizer)

# logger.info('Initializing tensorboard')
tensorboard_logs_dir = f'logs/tensorboard/{main_config.exp_name}/'
tensorboard_writer = SummaryWriter(tensorboard_logs_dir)

[2024-04-03 11:18:19,064: INFO] Loading dataset (load_codeparrot_dataset.py:7)


/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.w

In [18]:
# logger.info('Training')
model._hf_peft_config_loaded = True # silence warnings - for research it is ok
model.config.use_cache = False # silence warnings from torch

trainer_args = main_config.trainer_args

In [19]:
model.device

device(type='cuda', index=0)

In [20]:
# transformers.TrainingArguments(
#         per_device_train_batch_size=trainer_args.per_device_train_batch_size,
#         gradient_accumulation_steps=trainer_args.gradient_accumulation_steps,
#         warmup_steps=trainer_args.warmup_steps,
#         max_steps=trainer_args.max_steps,
#         learning_rate=trainer_args.learning_rate,
#         fp16=trainer_args.fp16,
#         logging_steps=trainer_args.logging_steps,
#         output_dir=f'checkpoints/{main_config.exp_name}/outputs',
#         report_to=trainer_args.report_to,
#         gradient_checkpointing=True,
#         gradient_checkpointing_kwargs={'use_reentrant':True}
#     ).device

In [21]:
model

LTM_GPT(
  (transformer): GPT2Model(
    (wte): Embedding(50272, 5120)
    (wpe): Embedding(2048, 5120)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-37): 38 x GPT2Block(
        (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): CastOutputToFloat(
    (0): Linear(in_features=5120, out_features=50272, bias=False)
  )
  (model_): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50

In [27]:
from transformers.integrations import TensorBoardCallback

with torch.autocast(device_type="cuda", dtype=torch.float16):
    trainer_ = transformers.Trainer(
        model=model, 
        train_dataset=dataset['train'],
        args=transformers.TrainingArguments(
            per_device_train_batch_size=trainer_args.per_device_train_batch_size,
            gradient_accumulation_steps=trainer_args.gradient_accumulation_steps,
            warmup_steps=trainer_args.warmup_steps,
            max_steps=trainer_args.max_steps,
            learning_rate=trainer_args.learning_rate,
            fp16=trainer_args.fp16,
            logging_steps=trainer_args.logging_steps,
            output_dir=f'checkpoints/{main_config.exp_name}/outputs',
            report_to=trainer_args.report_to,
            gradient_checkpointing=True,
            gradient_checkpointing_kwargs={'use_reentrant':True}
        ),
        data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
        callbacks=[TensorBoardCallback(tb_writer=tensorboard_writer)]
    )
    trainer_.train()

/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 100.00 MiB. GPU 0 has a total capacty of 23.69 GiB of which 23.94 MiB is free. Including non-PyTorch memory, this process has 23.65 GiB memory in use. Of the allocated memory 23.32 GiB is allocated by PyTorch, and 23.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF